In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import subprocess
import re
import os
import glob
import pandas as pd
import time
from time import sleep
import random
import pdb

In [ ]:

# Load the accounts DataFrame from accounts.csv
accounts_df = pd.read_csv('reference/accounts.csv')
# Read the list of already-scraped profiles from scraped_profiles.txt
with open('reference/scraped_profiles.txt', 'r') as file:
    scraped_profiles = file.read().split('\n')



In [ ]:
def login_linkedin(driver, username, password):
    # Navigate to LinkedIn, enter username and password, submit form
    # If verification page appears, call handle_verification
    # Navigate to the LinkedIn login page
    driver.get('https://www.linkedin.com/login')
    
    # Input username
    username_field = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, 'username')))
    username_field.send_keys(username)
    sleep(random.random()*3)    


    # Input password
    password_field = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, 'password')))
    password_field.send_keys(password)
    sleep(random.random()*3)    

    # Locate the sign in button
    sign_in_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '//button[@type="submit"]')))

    # Create an ActionChains object mouse movement to avoid detection
    actions = ActionChains(driver)

    # Move to the sign in button and click it
    actions.move_to_element(sign_in_button).click().perform()



def get_verification_code_from_file(download_dir='/Users/adamhunter/Documents/school projs/capstone/data'):
    # Get a list of all the pgp*.txt files in the download directory
    files = glob.glob(os.path.join(download_dir, 'pgp*.txt'))

    # Find the most recent file
    latest_file = max(files, key=os.path.getctime)

    # Open the file and read the contents
    with open(latest_file, 'r') as f:
        contents = f.read()

    # Use regex to find the verification code in the line that starts with "Subject:"
    match = re.search(r'Subject:.*?(\d{6})', contents)
    if match:
        verification_code = match.group(1)
    else:
        print("No verification code found in email.")
        verification_code = None

    return verification_code

def grab_verification(driver, username, password):
    # Save the handle of the original tab
    original_tab = driver.current_window_handle

    # Open a new tab
    driver.execute_script("window.open('');")

    # Switch to the new tab (it's always the last one)
    driver.switch_to.window(driver.window_handles[-1])

    # Navigate to ProtonMail
    driver.get('https://mail.protonmail.com/login')
    # Input username
    username_field = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, 'username')))
    sleep(random.random()*2)    
    username_field.send_keys(username)

    password_field = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, 'password')))
    sleep(random.random()*3)    
    password_field.send_keys(password)

    # Submit form
    login_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.button-large')))
    sleep(random.random()*3)    
    login_button.click()
    print('clicked login to email')

    # Click the first email in the inbox
    first_email = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.item-container-wrapper:nth-child(1) .item-subject .max-w100')))
    first_email.click()

    # Click 'More options' button
    more_options_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.button:nth-child(11)')))
    more_options_button.click()

    # Click 'View headers' button
    view_headers_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.dropdown-item:nth-child(10) .flex-item-fluid')))
    view_headers_button.click()

    # Click 'Download' button
    download_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.button-solid-norm:nth-child(2)')))
    download_button.click()

    

    # Switch back to the original tab
    driver.switch_to.window(original_tab)

    sleep(3)

    verification_code = get_verification_code_from_file()

    # Input verification code
    verification_field = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, 'input__email_verification_pin')))
    sleep(random.random()*3)    
    verification_field.send_keys(verification_code)
    sleep(random.random())

    # Click the verify button
    verify_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, 'email-pin-submit-button')))

    # Create an ActionChains object mouse movement to avoid detection
    actions = ActionChains(driver)

    # Move to the sign in button and click it
    actions.move_to_element(verify_button).click().perform()

def collect_links(driver, scraped_profiles, target_count=10):
    collected_links = set()

    while len(collected_links) < target_count:
        print(len(collected_links))
        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(1 + 2*random.random())

        # Scroll to the top and wait 1-3 seconds triggering infinite scroll basically
        driver.execute_script("window.scrollTo(0, 0);")
        sleep(1 + 2*random.random())

        # Collect all links
        all_links = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.TAG_NAME, "a")))
        member_links = [link.get_attribute('href') for link in all_links if link.get_attribute('href').startswith("https://www.linkedin.com/in/")]

        # Use regex to shorten the member links to cutoff anything after the profile slug
        member_links = [re.match("(https://www.linkedin.com/in/[^/]*/[^/]*)", link).group(1) for link in member_links]

        # Convert the list to a set to remove duplicates
        member_links = set(member_links)

        # Filter the member_links to include only those that are not in the scraped_profiles
        new_links = [link for link in member_links if link not in scraped_profiles]

        # Add the new links to the collected links set
        collected_links.update(new_links)

    # Convert the set back to a list and return it
    return list(collected_links)

def scrape_all(driver, member_links):
    for member_link in member_links:
        try:
            # Open the member link in a new tab
            driver.execute_script("window.open('');")
            driver.switch_to.window(driver.window_handles[-1])
            driver.get(member_link)

            # Find the "More actions" button using the aria-label attribute
            more_actions_button = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//button[@aria-label='More actions']")))

            # Use JavaScript to click the "More actions" button
            driver.execute_script("arguments[0].click();", more_actions_button)

            sleep(1 + random.random())

            save_to_pdf_option = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'artdeco-dropdown__item') and contains(@aria-label, 'Save')]")))

            save_to_pdf_option.click()

            sleep(1+3*random.random())

            # Close the current tab
            driver.close()

            # Switch back to the original tab
            driver.switch_to.window(driver.window_handles[0])
        except NoSuchElementException as e:
            print(e)
            continue

def create_driver(account):
    print('driver started')
    chrome_options = Options()
    chrome_options.add_argument(f"user-agent={account['user_agent']}")
    #chrome_options.add_argument(f"--proxy-server={account['proxy']}")
    prefs = {'download.default_directory' : '/Users/adamhunter/Documents/school projs/capstone/data'}
    chrome_options.add_experimental_option('prefs', prefs)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.maximize_window() #max size for consistency
    return driver

def start_vpn(config_file):
    # Get the OpenVPN username and password from the environment variables
    openvpn_username = os.environ['OPENVPN_USERNAME']
    openvpn_password = os.environ['OPENVPN_PASSWORD']
    sudo_password = os.environ['SUDO_PASSWORD']

    # Create a temporary file to store the credentials
    with open('credentials.txt', 'w') as f:
        f.write(f"{openvpn_username}\n{openvpn_password}\n")
    config_path = '/Users/adamhunter/Documents/school projs/capstone/reference/ProtonVPN_server_configs/'
    # Start OpenVPN with a specific configuration file
    command = ['sudo', '-S', 'openvpn', '--config', config_path + config_file, '--auth-user-pass', 'credentials.txt']
    process = subprocess.Popen(command, stdin=subprocess.PIPE, universal_newlines=True)
    # Send the password to the sudo prompt and disconnect the stdin pipe
    process.stdin.write(sudo_password + '\n')
    process.stdin.close()

    # Wait for the VPN to establish a connection
    time.sleep(10)

    # Delete the temporary credentials file
    os.remove('credentials.txt')

def main(accounts_df, scraped_profiles, starturl='https://www.linkedin.com/groups/7036558/members/'):
    for index, account in accounts_df.iterrows():
        # Skip the iteration if the account status is not 'active', move to next account
        if account['acc_status'] != 1:
            continue
        # start_vpn(account['proxy'])
        # Extracting the username and password from the selected account
        username = account['username']
        password = 'Linked00!'
        driver = create_driver(account)
        # Logging into LinkedIn with the selected account
        login_linkedin(driver, username, password)
        if "Let's do a quick verification" in driver.page_source:
            grab_verification(driver, username, password)
            # Get the verification code from the most recent pgp*.txt file
        elif "Let's do a quick security check" in driver.page_source: #captcha page
            driver.switch_to.window(driver.current_window_handle)
            sleep(30)
        if "Let's do a quick security check" in driver.page_source: #captcha page
            driver.quit()
            continue
        
        driver.get(starturl)
        # Collect links
        member_links = collect_links(driver, scraped_profiles, target_count=30)

        # Scrape all profiles
        scrape_all(driver, member_links)

        # Write the new member_links to the file
        with open('reference/scraped_profiles.txt', 'a') as file:
            for profile in member_links:
                file.write(f"{profile}\n")

        # Append member_links to scraped_profiles
        scraped_profiles.extend(member_links)
        driver.quit()
        # os.system('echo {} | sudo -S killall openvpn'.format(os.environ['SUDO_PASSWORD']))

    

In [ ]:
main(accounts_df, scraped_profiles, starturl='https://www.linkedin.com/groups/7036558/members/')